<a href="https://colab.research.google.com/github/Ayushmangal689/Summer_analytics/blob/main/capstone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
pip install pathway # Install Pathway if not already installed (use ! or %pip in notebooks)

In [45]:
# Import necessary libraries
import pandas as pd
import numpy as np
import itertools
import bokeh.plotting
import panel as pn

In [47]:
# Load dataset
df = pd.read_csv('dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [48]:
# Combine date and time into a single Timestamp column
df['Timestamp']= pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'] , format="%d-%m-%Y %H:%M:%S")

In [49]:
# Encode vehicle type with corresponding weights
vehicle_weight = {'bike': 0.5 ,'car':1.0,'truck':1.5}
df['VehicleWeight']=df['VehicleType'].map(vehicle_weight)

In [50]:
# Encode traffic conditions numerically
traffic_map={'low': 1,'medium':2, 'high':3}
df['TrafficLevel']=df['TrafficConditionNearby'].map(traffic_map)

In [51]:
# Sort by parking lot and timestamp
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [40]:
# Define Model 2 pricing function using demand-based factors
def model2_demand_based(df, base_price=10.0,
                        alpha=1.2, beta=0.8, gamma=0.5,
                        delta=1.0, epsilon=0.7, lambd=0.3):
    df = df.copy()
# Calculate raw demand score
    df['RawDemand'] = (
        alpha * (df['Occupancy'] / df['Capacity']) +
        beta * df['QueueLength'] -
        gamma * df['TrafficLevel'] +
        delta * df['IsSpecialDay'] +
        epsilon * df['VehicleWeight']
    )

# Normalize demand per parking lot
    df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
        lambda x: (x - x.min()) / (x.max() - x.min() + 1e-5))

# Compute final price with bounds
    df['Price_Model2'] = base_price * (1 + lambd * df['NormalizedDemand'])
    df['Price_Model2'] = df['Price_Model2'].clip(lower=base_price * 0.5, upper=base_price * 2.0)

    return df
# Apply model
df_model2 = model2_demand_based(df)

In [42]:
import pathway as pw  # Pathway real-time pipeline setup

# Define the schema for incoming streaming data
class ParkingSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str

# Load streaming CSV with schema and 500ms batch commit
t = pw.io.csv.read("e3b2e988-6f34-443c-bd4f-eb6fe9616d24.csv",
                  schema=ParkingSchema,
                  mode="streaming",
                  autocommit_duration_ms=500)

# Select relevant columns and calculate occupancy ratio
t = t.select(
    t.ID,
    t.SystemCodeNumber,
    t.Capacity,
    t.Occupancy,
    t.QueueLength,
    t.VehicleType,
    t.TrafficConditionNearby,
    t.IsSpecialDay,
    occupancy_ratio=t.Occupancy / t.Capacity
)

# Define a simple pricing function as a UDF
@pw.udf
def compute_price(base: float, occ_ratio: float) -> float:
    return base + 2.0 * occ_ratio

# Apply pricing UDF to stream
t = t.with_columns(
    Price=compute_price(10.0, t.occupancy_ratio)
)

# Output result
t = t.select(t.SystemCodeNumber, t.Price, t.occupancy_ratio)
pw.io.jsonlines.write(t, "results_stream.jsonl")

In [53]:
# Bokeh visualization for Model 2 pricing
from bokeh.layouts import row
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Category10
from bokeh.models import Legend

output_notebook()

p = figure(title="Dynamic Pricing for Parking Lots (Model 2)",
           x_axis_type="datetime",
           width=800, height=400,
           x_axis_label="Time", y_axis_label="Price ($)")

colors = Category10[10] + Category10[4]
legend_items = []

# Plot each lot with different color
for i, lot in enumerate(df_model2['SystemCodeNumber'].unique()):
    lot_df = df_model2[df_model2['SystemCodeNumber'] == lot]
    line = p.line(x=lot_df['Timestamp'],
                  y=lot_df['Price_Model2'],
                  line_width=2, color=colors[i % len(colors)])
    legend_items.append((lot, [line]))

# Add legend to the plot
legend = Legend(items=legend_items, location="center")
p.add_layout(legend, 'right')

# Show interactive plot
show(p)


